# Image Classification Using Bag-of-Words

## Part I:  Review of bag-of-words for image classiﬁcation

We will solve image classiﬁcation, with the relatively simple but eﬀective bag-of-words approach. We treat an image as a set of regions and ignore the spatial relationships between diﬀerent regions. The image classiﬁcation problem can then be solved by count the frequencies of diﬀerent regions appearing in an image. Image classiﬁcation using bag-of-words includes the following steps: 

* **Region Selection**: Select some regions in the images so that we can extract features from these regions in the next step. These regions can be selected by feature detection methods like edge or corner detection, or you can just uniformly sample the image. 

* **Feature Extraction**: We extract features from the selected regions. One commonly used feature is SIFT. We extract features from every image in the training set. These features are collected to compute the visual vocabulary for image representation. 

* **Building visual vocabulary**: Once we have the features extracted from training images, we build a visual vocabulary by grouping them together to form a few clusters. We will use k-means algorithm to group the features. The reason why we need this step is to reduce the redundancy in feature space and have a more concise feature representation of images. 

* **Learning and recognition**: Given the visual vocabulary, each training image is represented by a histogram, where the features in the image populate bins that correspond to the visual word closest to them. Thereafter, we will use k-nearest neighbors to perform classiﬁcation for a test image, also represented by a histogram using the same vocabulary.

# Part II: Simple face classifier

We will make a classiﬁer to tell whether there is a face in a given image. The dataset contains 200 images with faces and 200 images without faces. We will pick 100 images from each group for training and the other 100 images for testing. The skeleton code is given. Please write your code at the ”WRITE YOUR CODE HERE” prompt

In [ ]:
import glob
import random
import numpy as np
from matplotlib import pyplot as plt
import cv2

# Parameters
posData = 'images/face/'
negData = 'images/nonface/'
posTrainRatio = 0.5
negTrainRatio = 0.5
imSize = 133, 200 # resize each image to this size
nIntPts = 200 # maximum number of interest points to be extracted from an image
wGrid = 5 # width of the small grids for uniform sampling
patchSize = 11 # an odd number indicate patch size for image patch feature
nCluster = 50 # number of cluster in k-means algorithm

def listImage(dataRoot):
    fileList = glob.glob(str(dataRoot + '*.jpg'))
    imNum = len(fileList)
    fileList = [fileList[i].replace('\\', '/') for i in range(imNum)]
    return fileList

def loadImage(imgName, imSize):
    # load image, resize and make RGB to grayscale
    img = cv2.imread(imgName)
    img = cv2.resize(img, (imSize[1], imSize[0]))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img

def createSplit(imgList, ratio):
    random.shuffle(imgList)
    trainList = imgList[:round(len(imgList)*ratio)]
    testList = imgList[round(len(imgList)*ratio):]
    return trainList, testList

# Data preprocessing: Creaete Training / Testing Split
posList = listImage(posData)
negList = listImage(negData)
trainPosList, testPosList = createSplit(posList, posTrainRatio)
trainNegList, testNegList = createSplit(negList, negTrainRatio)
trainList = trainPosList + trainNegList
trainLabel = np.concatenate((np.ones(len(trainPosList)), np.zeros(len(trainNegList))))
testList = testPosList + testNegList
testLabel = np.concatenate((np.ones(len(testPosList)), np.zeros(len(testNegList))))
testPosLabel = np.ones(len(testPosList))
testNegLabel = np.zeros(len(testNegList))

## 1. Extract interest points

You are going to try two methods to extract interest points from image: 

* **(a) Uniformly sample the images.** You can divide the image into regular grids and choose a point in each grid and then uniformly choose nPts number of interest points.
* **(b) Sample on corners.** First use the Harris Corner detector to detect corners in the image and then uniformly choose nPts number of interest points. (This has already been implemented for you as an example.) 

In [ ]:
def uniformSampling(imSize, nPts, wGrid):
    ''' Uniformly sample the images.
    Args: 
        imSize: size of images (height, width)
        nPts: maximum number of interest points to be extracted
        wGrid: width of the small grids
    Return:
        pts: a list of interest points (Yi, Xi)
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return pts

def cornerSampling(img, nPts):
    '''
    Args: 
        img: image which you want to extract interest points
        nPts: maximum number of interest points to be extracted
    Return:
        pts: a list of interest points (Yi, Xi)
    '''
    dst = cv2.cornerHarris(img,2,3,0.04)
    dst = cv2.dilate(dst,None)
    cor = np.zeros(img.shape)
    cor[dst>0.01*dst.max()]=[1]
    ptsY, ptsX = np.where(cor==1)
    num = min(nPts, len(ptsY))
    choice = np.random.choice(len(ptsY), num, replace=False)
    ptsY = ptsY[choice]
    ptsX = ptsX[choice]
    pts = [(ptsY[i], ptsX[i]) for i in range(num)]
    return pts

def plotInterestPoints(img, pts, idx=0):
    ptsMap = np.zeros(img.shape)
    ptsY = [Y for Y, X in pts]
    ptsX = [X for Y, X in pts]
    ptsMap[ptsY, ptsX] = 1
    plt.figure(idx)
    plt.subplot(121),plt.imshow(img,cmap = 'gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(ptsMap,cmap = 'gray')
    plt.title('Interest Points Image'), plt.xticks([]), plt.yticks([])

# Here is the code for you to test your implementation
sampleImg = loadImage(trainList[0], imSize)
print('Interest Points extracted by uniform (above) and edge (below) method:')
ptsU = uniformSampling(imSize, nIntPts, wGrid)
plotInterestPoints(sampleImg, ptsU, idx=0)
ptsE = cornerSampling(sampleImg, nIntPts)
plotInterestPoints(sampleImg, ptsE, idx=1)

## 2. Extract Features
You are required to try two kinds of features. 
* **(a) SIFT feature.** Here we use the SIFT implementation in *OpenCV* package (this has already been implemented for you as an example). 
* **(b) Image Patch feature.** Extract a small image patch around each feature point. You can decide the size of each patch and how many pixels it should cover on you own. 

In [ ]:
def extractSIFTfeature(img, pts):
    '''
    Args:
        img: input image
        pts: detected interest points in the previous step
    Return:
        features: a list of SIFT descriptor features for each interest point
    '''
    sift = cv2.xfeatures2d.SIFT_create()
    kp = [cv2.KeyPoint(ptsX, ptsY, 1) for ptsY, ptsX in pts]
    _, des = sift.compute(img, kp)
    features = [des[i] for i in range(des.shape[0])]
    return features

def extractImagePatchfeature(img, pts, patchSize):
    '''
    Args:
        img: input image
        pts: detected interest points in the previous step
        patchSize: an odd number indicate patch size
    Return:
        features: a list of image patch features (1-d) for each interest point
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return features
    
# Here is the code for you to test your implementation
featSIFT = extractSIFTfeature(sampleImg, ptsE)
print('length of SIFT feature list', len(featSIFT))
print('dimension of feature', featSIFT[0].shape)
featPatch = extractImagePatchfeature(sampleImg, ptsE, patchSize)
print('length of Image Patch feature list', len(featPatch))
print('dimension of feature', featPatch[0].shape)

## 3. Build visual vocabulary
Use K-means clustering to form a visual vocabulary. You can use the python k-means package. You can decide the number of clusters yourself. The default number of cluster centers in k-means is 50. You can try diﬀerent metrics to do k-means clustering and see which gives the best results. 

In [ ]:
from sklearn.cluster import KMeans

def getImgFeat(img, imSize, nIntPts, wGrid, patchSize, ptType, featType):
    ''' Output a list of detected features for an image
    Args: 
        img: image which you want to extract interest points
        imSize: size of images (height, width)
        nIntPts: maximum number of interest points to be extracted
        wGrid: width of the small grids
        patchSize: an odd number indicate patch size
        ptType: 'uniform' or 'corner' indicates the interest pts sampling method
        featType: 'sift' or 'patch' indicates the feature extraction method
    Return:
        extractFeatList: a list of image patch features (1-d) for each interest point
    '''
    if ptType == 'uniform':
        intPts = uniformSampling(imSize, nIntPts, wGrid)
    elif ptType == 'corner':
        intPts = cornerSampling(img, nIntPts)
    else:
        assert False, 'ptType must be either uniform or corner'
        
    if featType == 'sift':
        extractFeatList = extractSIFTfeature(img, intPts)
    elif featType == 'patch':
        extractFeatList = extractImagePatchfeature(img, intPts, patchSize)
    else:
        assert False, 'featType must be either sift or patch'
        
    return extractFeatList

def collectFeat(trainList, imSize, nIntPts, wGrid, patchSize, ptType, featType):
    ''' Collect extracted features for each image among training data
    Args:
        trainList: list of images filepath
        imSize: size of images (height, width)
        nIntPts: maximum number of interest points to be extracted
        wGrid: width of the small grids
        patchSize: an odd number indicate patch size
        ptType: 'uniform' or 'corner' indicates the interest pts sampling method
        featType: 'sift' or 'patch' indicates the feature extraction method
    Return:
        feats: (# of features, dim of feature) array
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return feats

def formVisualVocab(feats, nCluster):
    ''' Use k-means algorithm to find k cluster centers, output k-means model
    Args: 
        feats: list of features collected from training data
        nCluster: number of cluster in k-means algorithm
    Return:
        model: k-means model for following steps
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return model

# Here is the code for you to test your implementation
feats = collectFeat(trainList, imSize, nIntPts, wGrid, patchSize, ptType='uniform', featType='sift')
print(feats.shape) # should be (total number of extracted features, dim of feature)
model = formVisualVocab(feats, nCluster)
centers = model.cluster_centers_
print(centers.shape) # should be (nCluster, dim of feature)

## 4. Compute histogram representation
Compute the histogram representation of each image, with bins deﬁned over the visual words in the vocabulary. These histograms are the bag-of-words representations of images that will be used for image classiﬁcation. 

In [ ]:
def getHistogram(img, model, nCluster, imSize, nIntPts, wGrid, patchSize, ptType, featType):
    '''
    Calculate histogram representation for single image
    Args:
        img: input image
        model: k-means model from previous step
        nCluster: number of cluster in k-means algorithm
        imSize: size of images (height, width)
        nIntPts: maximum number of interest points to be extracted
        wGrid: width of the small grids
        patchSize: an odd number indicate patch size
        ptType: 'uniform' or 'corner' indicates the interest pts sampling method
        featType: 'sift' or 'patch' indicates the feature extraction method
    Return:
        hist: histogram representation (1-d) for input image
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return hist
    
def computeHistograms(trainList, model, nCluster, imSize, nIntPts, wGrid, patchSize, ptType, featType):
    ''' Compute histogram representation for whole training dataset
    Args: 
        trainList: list of images filepath
        model: k-means model from formVisualVocab
        nCluster: number of cluster in k-means algorithm
        imSize: size of images (height, width)
        nIntPts: maximum number of interest points to be extracted
        wGrid: width of the small grids
        patchSize: an odd number indicate patch size
        ptType: 'uniform' or 'corner' indicates the interest pts sampling method
        featType: 'sift' or 'patch' indicates the feature extraction method
    Return:
        hists: (# of images, nCluster) array - histogram representations among training dataset
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return hists
    
# Here is the code for you to test your implementation
hist = getHistogram(sampleImg, model, nCluster, imSize, nIntPts, wGrid, patchSize, ptType='uniform', featType='sift')
print(hist.shape) # should be (nCluster)
hists = computeHistograms(trainList, model, nCluster, imSize, nIntPts, wGrid, patchSize, ptType='uniform', featType='sift')
print(hists.shape) # should be (# of training images, nCluster)

## 5. K nearest neighbor classifier
After building the visual vocabulary, we now do image classiﬁcation using the nearest neighbors method. Given a new image, ﬁrst represent it using the visual vocabulary and then ﬁnd the closest representation in the training set. The test image can be assumed to be in the same category as its nearest neighbors in the training set. To make the algorithm more robust, you can also try to ﬁnd the ﬁrst K-nearest (K =3 or 5) neighbors. You may try diﬀerent distance metrics when doing K-nearest neighbors search.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def KNNclassifier(trainX, trainY, n_neighbors):
    ''' Return a KNN model by fitting training data (trainX, trainY)
    Args:
        trainX: a (# of images, nCluster) array of BoW features for training data
        trainY: a (# of images) array of class label for training data
        n_neighbors: # of neighbors used in KNN classifier
    Return:
        model: KNN classifier model
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return model

def getAccuracy(testX, testY, model):
    ''' Output the testing accuracy for KNN classifier model 
    Args:
        testX: a (# of images, nCluster) array of BoW features for testing data
        testY: a (# of images) array of class label for testing data
        model: KNN classifier model
    Return:
        accu: accuracy of classification prediction on testing data
    '''
    #-------------------------------------#
    #         WRITE YOUR CODE HERE        #
    #-------------------------------------#
    return accu

# Here is the code for you to test your implementation
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
model = KNNclassifier(X, y, n_neighbors=3)
print(model.predict([[1.1]])) # Should be [0]
Xp = [[0.5], [2.5]]
Yp = [0, 0]
acc = getAccuracy(Xp, Yp, model)
print(acc) # Should be 0.5

## 6. Calculate testing accuracy
Finally, we can calculate the testing accuracy of our face classifier. The accuracy should be organized into a 2D table as follows. 

|      .       |  Uniform |  Uniform |  Corner  |  Corner  |
|:------------:|:--------:|:--------:|:--------:|:--------:|
|      .       | Positive | Negative | Positive | Negative |
| SIFT Feature |     .    |     .    |     .    |     .    |
|  Image Patch |     .    |     .    |     .    |     .    |

Some of the methods may have poor accuracy. But that is ﬁne, don’t worry too much about accuracy. You will get full credit as long as you can implement and reason about the various methods. 

In [ ]:
#-------------------------------------#
#         WRITE YOUR CODE HERE        #
#-------------------------------------#

 ## 7. Explain your implementation
 Brieﬂy explain how you implemented the methods and which parameters you ﬁnd to have a signiﬁcant impact on the ﬁnal performance. 

Ans: